### Alternative macroeconomic datasets

In [1]:
import os
os.getcwd()
# os.chdir(path)    # or you can set your working dir.

'/Users/xingfuxu/PycharmProjects/EquityPremiumPredictionML-Jupyter'

In [2]:
# Your working dir should include "NN_models.py", Perform_CW_test.py" and "Perform_PT_test.py" files.
from Perform_CW_test import CW_test
from Perform_PT_test import PT_test
from NN_models import Net2, Net4

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid
import torch
from skorch import NeuralNetRegressor
from tqdm import tqdm
#
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import minmax_scale
import warnings

In [4]:
# set seed
torch.manual_seed(1)
np.random.seed(1)

# read data
predictor_df = pd.read_excel(open('ml_equity_premium_data.xlsx', 'rb'), sheet_name='result_predictor')
predictor_df.head()

# read fred data
fred_md0 = pd.read_excel(open('ml_equity_premium_data.xlsx', 'rb'), sheet_name='FRED_MD')
fred_md = fred_md0.T.dropna().T
fred_md  # range from 1959:01 to 2020:12

,sasdate,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,...,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,MZMSL,DTCOLNVHFNM,DTCTHFNM,INVEST
0,1/1/1959,2437.296,2288.8,17.302,292258.8329,18235.77392,22.625,23.4581,22.1904,32.4078,...,56.918,17.791,11.358,2.13,2.45,2.04,274.9,6476.0,12298.0,84.2043
1,2/1/1959,2446.902,2297.0,17.482,294429.5453,18369.56308,23.0681,23.7747,22.3827,32.6455,...,56.951,17.798,11.375,2.14,2.46,2.05,276.0,6476.0,12298.0,83.528
2,3/1/1959,2462.689,2314.0,17.647,293425.3813,18523.05762,23.4004,23.9186,22.4925,32.6455,...,57.022,17.785,11.395,2.15,2.45,2.07,277.4,6508.0,12349.0,81.6405
3,4/1/1959,2478.744,2330.3,17.584,299331.6505,18534.466,23.8989,24.2641,22.8221,33.1606,...,57.08,17.796,11.436,2.16,2.47,2.08,278.1,6620.0,12484.0,81.8099
4,5/1/1959,2493.228,2345.8,17.796,301372.9597,18679.66354,24.2589,24.4655,23.0418,33.319,...,57.175,17.777,11.454,2.17,2.48,2.08,280.1,6753.0,12646.0,80.7315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,8/1/2020,17603.151,13900.7,117.335,1554439.0,543404.0,102.8885,99.8433,99.5522,105.0636,...,86.66,98.922,120.338,25.51,29.39,22.87,21113.5,344302.07,726966.76,4362.8621
740,9/1/2020,17698.892,14029.2,118.656,1564146.0,552767.0,102.7766,99.3909,98.7596,103.9916,...,86.611,98.659,120.743,25.49,29.09,23.01,21271.5,345629.27,728899.98,4417.394
741,10/1/2020,17569.407,14119.1,118.886,1572531.0,552193.0,103.9494,100.6768,99.7284,104.8031,...,86.515,98.558,120.902,25.58,29.4,22.99,21365.5,348519.86,730630.92,4492.1138
742,11/1/2020,17350.354,14015.9,118.173,1569904.0,545248.0,104.8525,101.3136,100.5163,105.1711,...,86.295,98.704,120.929,25.69,29.55,23.1,21576.9,350858.18,733193.14,4605.6462


In [5]:
start_month = predictor_df.index[predictor_df['month'] == 195901][0]
predictor_df = predictor_df.iloc[start_month:, :]
predictor_df.reset_index(inplace=True, drop=True)
predictor_df.head()
#
fred_md.iloc[:, 1:] = minmax_scale(fred_md.iloc[:, 1:])
tech_ind = predictor_df.columns.get_loc('MA_1_9')
predictor0 = pd.concat([predictor_df.loc[:, ['log_equity_premium']],
                        fred_md.iloc[:, 1:], predictor_df.iloc[:, 15:]],axis=1)
predictor0

,log_equity_premium,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,...,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_1,MOM_2,MOM_3,MOM_6,MOM_9,MOM_12
0,0.004112,0.0,0.0,0.0,0.008774,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
1,0.004590,0.000575,0.000681,0.001721,0.010454,0.00025,0.005039,0.003616,0.002224,0.002904,...,1,1,1,1,0,1,1,1,1,1
2,0.001446,0.001519,0.002093,0.003299,0.009677,0.000537,0.008819,0.005259,0.003494,0.002904,...,1,1,1,1,1,1,1,1,1,1
3,0.037071,0.00248,0.003447,0.002696,0.01425,0.000559,0.014488,0.009205,0.007307,0.009198,...,1,1,1,1,1,1,1,1,1,1
4,0.021357,0.003346,0.004734,0.004723,0.01583,0.00083,0.018583,0.011505,0.009848,0.011134,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,0.069491,0.907294,0.9645,0.956412,0.985993,0.982484,0.912847,0.872319,0.894863,0.887766,...,1,1,1,1,1,1,1,1,1,1
740,-0.038997,0.913022,0.975173,0.969042,0.993508,1.0,0.911574,0.867152,0.885695,0.874667,...,1,1,1,1,0,1,1,1,1,1
741,-0.026865,0.905275,0.98264,0.971241,1.0,0.998926,0.924912,0.881837,0.896901,0.884583,...,1,1,1,1,0,0,0,1,1,1
742,0.103719,0.89217,0.974068,0.964424,0.997966,0.985933,0.935183,0.88911,0.906015,0.889079,...,1,1,1,1,1,1,1,1,1,1


In [6]:
# get all the predictors and set the log equity premium 1-month ahead
predictor = np.concatenate([predictor0['log_equity_premium'][1:].values.reshape(-1, 1),
                            predictor0.iloc[0:(predictor0.shape[0] - 1), 1:]], axis=1)
predictor = predictor.astype(float)
# number of rows
N = predictor.shape[0]

# number of all columns, including the log equity premium
n_cols = predictor.shape[1]

# Actual one-month ahead log equity premium
actual = predictor[:, [0]]

# Historical average forecasting as benchmark
y_pred_HA = predictor0['log_equity_premium'].values[0:(predictor0.shape[0] - 1), ].cumsum() / np.arange(1, N + 1)
y_pred_HA = y_pred_HA.reshape(-1, 1)

In [7]:
## Out-of-sample: 1990:01-2020:12
in_out_1990 = predictor_df.index[predictor_df['month'] == 199001][0]
actual_1990 = actual[in_out_1990:, ]
y_pred_HA_1990 = y_pred_HA[in_out_1990:, ]
MSFE_HA_1990 = mean_squared_error(y_pred_HA_1990, actual_1990)

# Machine Learning methods used in GKX (2020)
y_pred_PLS_1990, y_pred_PCR_1990,  y_pred_LASSO_1990 = [], [], []
y_pred_ENet_1990, y_pred_RF_1990 = [], []
y_pred_NN2_1990, y_pred_NN4_1990 = [], []

## Other commonly used machine learning method
y_pred_Ridge_1990 = []

In [8]:
# control the update month of models during out-of-sample period. 
month_index = 1  # We update our models annually, meaning we refresh them in months 1, 13, 25, ...

In [9]:
for t in tqdm(range(in_out_1990, N)):
    X_train_all = predictor[:t, 1:n_cols]
    y_train_all = predictor[:t, 0]
    # set 15% of all the train data as validation set
    X_train = X_train_all[0:int(len(X_train_all) * 0.85), :]
    X_validation = X_train_all[int(len(X_train_all) * 0.85):t, :]
    y_train = y_train_all[0:int(len(X_train_all) * 0.85)]
    y_validation = y_train_all[int(len(X_train_all) * 0.85):t]
    #
    warnings.filterwarnings('ignore')
    #
    if month_index % 12 == 1:
        month_index += 1
        # PLS
        PLS_param = {'n_components': [1, 2, 3, 4, 5, 6, 7, 8]}
        PLS_result = {}
        for param in ParameterGrid(PLS_param):
            PLS = PLSRegression(**param)
            PLS.fit(X_train, y_train)
            mse = mean_squared_error(PLS.predict(X_validation), y_validation)
            PLS_result[str(param)] = mse

        PLS_best_param = eval(min(PLS_result, key=PLS_result.get))
        PLS_model = PLSRegression(**PLS_best_param)
        PLS_model.fit(X_train_all, y_train_all)
        y_pred_PLS_1990.append(PLS_model.predict(predictor[[t], 1:n_cols])[0][0])

        # PCR
        PCR_param = {'n_components': [1, 2, 3, 4, 5, 6, 7, 8]}
        PCR_result = {}
        for param in ParameterGrid(PCR_param):
            pca = PCA(**param)
            pca.fit(X_train)
            comps = pca.transform(X_train)
            forecast = LinearRegression()
            forecast.fit(comps, y_train)
            mse = mean_squared_error(forecast.predict(pca.transform(X_validation)), y_validation)
            PCR_result[str(param)] = mse
        #
        PCR_best_param = eval(min(PCR_result, key=PCR_result.get))
        #
        PCR_model = PCA(**PCR_best_param)
        PCR_model.fit(X_train_all)
        PCR_comps = PCR_model.transform(X_train_all)
        PCR_forecast = LinearRegression()
        PCR_forecast.fit(PCR_comps, y_train_all)
        y_pred_PCR_1990.append(PCR_forecast.predict(PCR_model.transform(predictor[[t], 1:n_cols]))[0])

        # LASSO
        LASSO_param = {'alpha': list(10 ** np.arange(-4, 1 + 0.001, 0.2))}
        LASSO_result = {}
        for param in ParameterGrid(LASSO_param):
            LASSO = Lasso(**param)
            LASSO.fit(X_train, y_train)
            mse = mean_squared_error(LASSO.predict(X_validation), y_validation)
            LASSO_result[str(param)] = mse
        #
        LASSO_best_param = eval(min(LASSO_result, key=LASSO_result.get))
        #
        LASSO_model = Lasso(**LASSO_best_param)
        LASSO_model.fit(X_train_all, y_train_all)
        y_pred_LASSO_1990.append(LASSO_model.predict(predictor[[t], 1:n_cols])[0])

        # ENet
        ENet_param = {'alpha': list(10 ** np.arange(-4, 1 + 0.001, 0.2)),
                      'l1_ratio': list(np.arange(0.2, 1, 0.3))}
        ENet_result = {}
        for param in ParameterGrid(ENet_param):
            ENet = ElasticNet(**param)
            ENet.fit(X_train, y_train)
            mse = mean_squared_error(ENet.predict(X_validation), y_validation)
            ENet_result[str(param)] = mse

        ENet_best_param = eval(min(ENet_result, key=ENet_result.get))
        ENet_model = ElasticNet(**ENet_best_param)
        ENet_model.fit(X_train_all, y_train_all)
        y_pred_ENet_1990.append(ENet_model.predict(predictor[[t], 1:n_cols])[0])


        # RF
        RF_param = {'n_estimators': [10, 50, 100, 150, 200],
                    'max_depth': [2, 3, 4],
                    'min_samples_leaf': [1, 3, 5]}
        RF_result = {}
        for param in ParameterGrid(RF_param):
            RF = RandomForestRegressor(**param)
            RF.fit(X_train, y_train)
            mse = mean_squared_error(RF.predict(X_validation), y_validation)
            RF_result[str(param)] = mse

        RF_best_param = eval(min(RF_result, key=RF_result.get))
        RF_model = RandomForestRegressor(**RF_best_param)
        RF_model.fit(X_train_all, y_train_all)
        y_pred_RF_1990.append(RF_model.predict(predictor[[t], 1:n_cols])[0])

        # Neural Network Models: NN2 & NN4
        X_train_all_tensor = torch.tensor(X_train_all, dtype=torch.float)
        y_train_all_tensor = torch.tensor(y_train_all.reshape(-1, 1), dtype=torch.float)
        X_train_tensor = torch.tensor(X_train, dtype=torch.float)
        y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float)
        X_validation_tensor = torch.tensor(X_validation, dtype=torch.float)
        y_validation_tensor = torch.tensor(y_validation.reshape(-1, 1), dtype=torch.float)

        # NN2
        NN2_result = {}
        NN2_architecture = {"module__n_feature": X_train_tensor.shape[1],
                            "module__n_hidden1": 32, "module__n_hidden2": 16,
                            "module__n_output": 1}
        NN2_param = {'module__dropout': [0.2, 0.4, 0.6, 0.8],
                    'lr': [0.001, 0.01],
                    'optimizer__weight_decay': [0.1, 0.01, 0.001]}
        for param in ParameterGrid(NN2_param):
            NN2 = NeuralNetRegressor(Net2, verbose=0, max_epochs=200,
                                     optimizer=torch.optim.SGD,
                                     **NN2_architecture, **param)
            NN2.fit(X_train_tensor, y_train_tensor)
            mse = mean_squared_error(NN2.predict(X_validation_tensor), y_validation)
            NN2_result[str(param)] = mse
        
        #
        NN2_best_param = eval(min(NN2_result, key=NN2_result.get))
        NN2_model = NeuralNetRegressor(Net2, verbose=0, max_epochs=200, optimizer=torch.optim.SGD,
                                       **NN2_architecture, **NN2_best_param)
        NN2_model.fit(X_train_all_tensor, y_train_all_tensor)
        y_pred_NN2_1990.append(NN2_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        #


        # NN4
        NN4_result = {}
        NN4_architecture = {"module__n_feature": X_train_tensor.shape[1],
                            "module__n_hidden1": 32, "module__n_hidden2": 16,
                            "module__n_hidden3": 8,  "module__n_hidden4": 4,
                            "module__n_output": 1}
        NN4_param = {'module__dropout': [0.2, 0.4, 0.6, 0.8],
                     'lr': [0.001, 0.01],
                     'optimizer__weight_decay': [0.1, 0.01, 0.001]}
        for param in ParameterGrid(NN4_param):
            NN4 = NeuralNetRegressor(Net4, verbose=0, max_epochs=200,
                                     optimizer=torch.optim.SGD,
                                     **NN4_architecture, **param)
            NN4.fit(X_train_tensor, y_train_tensor)
            mse = mean_squared_error(NN4.predict(X_validation_tensor), y_validation)
            NN4_result[str(param)] = mse

        #
        NN4_best_param = eval(min(NN4_result, key=NN4_result.get))
        NN4_model = NeuralNetRegressor(Net4, verbose=0, max_epochs=200, optimizer=torch.optim.SGD,
                                       **NN4_architecture, **NN4_best_param)
        NN4_model.fit(X_train_all_tensor, y_train_all_tensor)
        y_pred_NN4_1990.append(NN4_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])


        ## Other commmonly used ML methods
        # Ridge
        Ridge_param = {'alpha': list(10 ** np.arange(0, 20 + 0.001, 1))}
        Ridge_result = {}
        for param in ParameterGrid(Ridge_param):
            RIDGE = Ridge(**param)
            RIDGE.fit(X_train, y_train)
            mse = mean_squared_error(RIDGE.predict(X_validation), y_validation)
            Ridge_result[str(param)] = mse
        #
        Ridge_best_param = eval(min(Ridge_result, key=Ridge_result.get))
        Ridge_model = Ridge(**Ridge_best_param)
        Ridge_model.fit(X_train_all, y_train_all)
        y_pred_Ridge_1990.append(Ridge_model.predict(predictor[[t], 1:n_cols])[0])

    else:
        month_index += 1
        y_pred_PLS_1990.append(PLS_model.predict(predictor[[t], 1:n_cols])[0][0])
        y_pred_PCR_1990.append(PCR_forecast.predict(PCR_model.transform(predictor[[t], 1:n_cols]))[0])
        y_pred_LASSO_1990.append(LASSO_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_ENet_1990.append(ENet_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_RF_1990.append(RF_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_NN2_1990.append(NN2_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        y_pred_NN4_1990.append(NN4_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        # Other commonly used ML methods
        y_pred_Ridge_1990.append(Ridge_model.predict(predictor[[t], 1:n_cols])[0])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 371/371 [51:12<00:00,  8.28s/it]


In [10]:
y_ml_pred_1990 = pd.DataFrame(np.array([y_pred_PLS_1990, y_pred_PCR_1990, y_pred_LASSO_1990,
                                   y_pred_ENet_1990, y_pred_RF_1990, y_pred_NN2_1990,  
                                   y_pred_NN4_1990, y_pred_Ridge_1990]),
                         index=['PLS', 'PCR', 'LASSO', 'ENet', 
                                'RF', 'NN2', 'NN4', 'Ridge'],
                         columns=predictor_df.month[in_out_1990:N]).T

In [11]:
# Performance compared with HA benchmark

def compute_oos_r_square(actual, y_benchmark, y_pred):
    MSFE_benchmark = mean_squared_error(y_benchmark, actual)
    MSFE_pred = mean_squared_error(y_pred, actual)
    return 1 - MSFE_pred / MSFE_benchmark


ml_oos_performance_macro = []

for col in y_ml_pred_1990.columns:
    oos_r_square = compute_oos_r_square(actual_1990, y_pred_HA_1990, y_ml_pred_1990[[col]].to_numpy())
    MSFE_adjusted, pvalue_MSFE = CW_test(actual_1990, y_pred_HA_1990, y_ml_pred_1990[[col]].to_numpy())
    success_ratio, PT_stat, pvalue_PT = PT_test(actual_1990, y_ml_pred_1990[[col]].to_numpy())
    ml_oos_performance_macro.append([oos_r_square * 100, MSFE_adjusted, pvalue_MSFE, success_ratio * 100, PT_stat, pvalue_PT])


ml_oos_performance_df_macro = pd.DataFrame(np.array(ml_oos_performance_macro),
                                          index=y_ml_pred_1990.columns,
                                          columns=['oos_r_square', 'MSFE_adjusted', 'pvalue_MSFE',
                                                   'success_ratio', 'PT_stat', 'pvalue_PT'])
# success ratio of HA
success_ratio_HA_1990, PT_HA_1990, p2_HA_1990 = PT_test(actual_1990, y_pred_HA_1990)
ml_oos_performance_df_macro.loc['HA'] = [0, np.nan, np.nan, success_ratio_HA_1990 * 100, PT_HA_1990, p2_HA_1990]

In [12]:
# save
import openpyxl
with pd.ExcelWriter("ml_equity_premium_robust_checks.xlsx", engine='openpyxl', mode='a') as writer:
    ml_oos_performance_df_macro.to_excel(writer, sheet_name='Alternative_macro_datasets')
ml_oos_performance_df_macro

,oos_r_square,MSFE_adjusted,pvalue_MSFE,success_ratio,PT_stat,pvalue_PT
PLS,-12.780186,-1.414876,0.921448,57.951482,-0.838793,0.799207
PCR,-2.656288,-0.088368,0.535208,59.299191,-2.259486,0.988073
LASSO,-3.819944,-2.137887,0.983737,57.412399,-2.172524,0.985092
ENet,-3.525860,-0.941180,0.826694,57.412399,-1.439037,0.924930
RF,-21.424013,-1.037624,0.850278,53.908356,0.918448,0.179192
NN2,-11.775560,-0.830699,0.796928,56.334232,-0.408842,0.658672
NN4,-17.873034,-0.024890,0.509929,64.150943,1.040546,0.149043
Ridge,0.120724,1.184863,0.118036,63.611860,0.674617,0.249959
HA,0.000000,NaN,NaN,64.150943,NaN,NaN
